# Create a Monopod Environment for RL

In [1]:
import sys
sys.executable

sys.path

['C:\\Users\\Nick\\Google Drive\\PythonPrograms\\machine_learning_projects\\ReinforcementLearning_Monopod',
 'C:\\Users\\Nick\\AppData\\Local\\Continuum\\anaconda3\\envs\\envCartPole\\python37.zip',
 'C:\\Users\\Nick\\AppData\\Local\\Continuum\\anaconda3\\envs\\envCartPole\\DLLs',
 'C:\\Users\\Nick\\AppData\\Local\\Continuum\\anaconda3\\envs\\envCartPole\\lib',
 'C:\\Users\\Nick\\AppData\\Local\\Continuum\\anaconda3\\envs\\envCartPole',
 '',
 'C:\\Users\\Nick\\AppData\\Roaming\\Python\\Python37\\site-packages',
 'C:\\Users\\Nick\\AppData\\Local\\Continuum\\anaconda3\\envs\\envCartPole\\lib\\site-packages',
 'C:\\Users\\Nick\\AppData\\Local\\Continuum\\anaconda3\\envs\\envCartPole\\lib\\site-packages\\IPython\\extensions',
 'C:\\Users\\Nick\\.ipython']

In [2]:
# %reset
# DERIVE THE EOMs SYMBOLICALLY ----------------------------------------------------------------------------------------------

# import libraries
import sympy as sym
import numpy as np

sym.init_printing()
from IPython.display import display #for pretty printing

# create symbolic variables

# system parameters
g = sym.symbols('g')
mb,ml1,ml2 = sym.symbols(['m_{body}','m_{leg1}','m_{leg2}']) # mass
lb,ll1,ll2 = sym.symbols(['l_{body}','l_{leg1}','l_{leg2}']) # length
Inb,Inl1,Inl2 = sym.symbols(['I_{body}','I_{leg1}','I_{leg2}']) # moment of intertia

# generalized coordinates
x,y,thb,thlt,thlb = sym.symbols(['x','y','\\theta_{body}','\\theta_{legtop}','\\theta_{legbot}']) 
dx,dy,dthb,dthlt,dthlb = sym.symbols(['\dot{x}','\dot{y}','\dot{\\theta}_{body}','\dot{\\theta}_{legtop}','\dot{\\theta}_{legbot}']) 
ddx,ddy,ddthb,ddthlt,ddthlb = sym.symbols(['\ddot{x}','\ddot{y}','\ddot{\\theta}_{body}','\ddot{\\theta}_{legtop}','\ddot{\\theta}_{legbot}']) 

q = sym.Matrix([[x],[y],[thb],[thlt],[thlb]])
dq = sym.Matrix([[dx],[dy],[dthb],[dthlt],[dthlb]])
ddq = sym.Matrix([[ddx],[ddy],[ddthb],[ddthlt],[ddthlb]])

# forces
# total joint action = actuator + rebound, but that will be dealt with elsewhere
tauh,tauk,GRFx,GRFy = sym.symbols(['\\tau_{hip}','\\tau_{knee}','G_x','G_y']) 

# STEP 1: position vectors ri = [x,y,theta] (world frame)
rb = sym.Matrix([[x],
                [y],
                [thb]])

rl1 = sym.Matrix([[x + 0.5*ll1*sym.sin(thb + thlt)],
                [y - 0.5*ll1*sym.cos(thb + thlt)],
                [thb + thlt]])

rl2 = sym.Matrix([[x + ll1*sym.sin(thb + thlt)+0.5*ll2*sym.sin(thb + thlt + thlb)],
                [y - ll1*sym.cos(thb + thlt) - 0.5*ll2*sym.cos(thb + thlt + thlb)],
                [thb + thlt + thlb]])

# the Jacobians
Jb = rb.jacobian(q)
Jl1 = rl1.jacobian(q)
Jl2 = rl2.jacobian(q)

# STEP 2: generate expressions for the system space velocities from the jacobians
vb = Jb*dq
vl1 = Jl1*dq
vl2 = Jl2*dq

# STEP 3: generate expressions for the kinetic and potential energy
# mass vectors
Mb = sym.Matrix([[mb,mb,Inb]])
Ml1 = sym.Matrix([[ml1,ml1,Inl1]])
Ml2 = sym.Matrix([[ml2,ml2,Inl2]])

T = 0.5*Mb*sym.matrix_multiply_elementwise(vb,vb) + 0.5*Ml1*sym.matrix_multiply_elementwise(vl1,vl1) + 0.5*Ml2*sym.matrix_multiply_elementwise(vl2,vl2)
T = T[0]
V = mb*g*rb[1] + ml1*g*rl1[1] + ml2*g*rl2[1]


# STEP 4: calculate each term of the Lagrange equation
# term 1
Lg1 = sym.zeros(len(q),1)
for i in range(len(q)):
    dT_ddq = sym.Matrix([sym.diff(T,dq[i])]) # get partial of T in dq_i
    Lg1[i] = dT_ddq.jacobian(q)*dq + dT_ddq.jacobian(dq)*ddq #...then get time derivative of that partial

# term 3
Lg3 = sym.Matrix([T]).jacobian(q).transpose() # partial of T in q

# term 4
Lg4 = sym.Matrix([V]).jacobian(q).transpose() # partial of U in q

# STEP 5: generalized forces

# force vectors for each link
tau_b = sym.Matrix([[0],[0],[-tauh]])
tau_l1 = sym.Matrix([[0],[0],[tauh-tauk]])
tau_l2 = sym.Matrix([[0],[0],[tauk]])

GRF_l2 = sym.Matrix([[GRFx],[GRFy],[0.5*ll2*GRFx*sym.cos(thb+thlt+thlb)+0.5*ll2*GRFy*sym.sin(thb+thlt+thlb)]])

Q = sym.zeros(len(q),1)
for j in range(len(q)):
    Q[j] = tau_b.transpose()*Jb[:,j]+(tau_l1).transpose()*Jl1[:,j]+(tau_l2+GRF_l2).transpose()*Jl2[:,j]

# AND combine!
EOM = Lg1 - Lg3 + Lg4 - Q

EOMs = sym.zeros(len(q),1)
for j in range(len(q)):
    EOMs[j] = EOM[j].simplify()

In [40]:
EOMs

⎡                                                                             
⎢                           -1.0⋅Gₓ + 0.5⋅\ddot{\theta}_{body}⋅l_{leg1}⋅m_{leg
⎢                                                                             
⎢                                                                             
⎢ -1.0⋅G_y + 0.5⋅\ddot{\theta}_{body}⋅l_{leg1}⋅m_{leg1}⋅sin(\theta_{body} + \t
⎢                                                                             
⎢                                                                             
⎢-1.0⋅Gₓ⋅l_{leg1}⋅cos(\theta_{body} + \theta_{legtop}) - 1.0⋅Gₓ⋅l_{leg2}⋅cos(\
⎢                                                                             
⎢                                                                             
⎢         -1.0⋅Gₓ⋅l_{leg1}⋅cos(\theta_{body} + \theta_{legtop}) - 1.0⋅Gₓ⋅l_{le
⎢                                                                             
⎢                                                   

In [3]:
# Position of monopod foot

pfoot = sym.Matrix([[x + ll1*sym.sin(thb + thlt) + ll2*sym.sin(thb + thlt + thlb)],
                    [y - ll1*sym.cos(thb + thlt) - ll2*sym.cos(thb + thlt + thlb)]])

pfoot

In [33]:
# Create an environment class for the monopod 
# functions: 
#   new_state, reward, terminated = env.step(previous_state, action)
#   env.plot(new_state)

class makeMonoEnv:
    """The monopod environment used for RL"""

    def __init__(self):
        self.mb = 5.0
        self.mlt = 2.5
        self.mlb = 2.5
        self.lb = 1.0
        self.llt = 0.5
        self.llb = 0.5
    
        self.Inb = 1/12*self.mb*self.lb**2
        self.Inlt = 1/12*self.mlt*self.llt**2
        self.Inlb = 1/12*self.mlb*self.llb**2
        
    def step(self, state, action):
        """step the RL model forward. 
        States should be [qx,... , qthlb, dqx,... , dqthlb, hm]
        Actions should be [tau_lt, tau_lb]"""
        
        qx_ = state[0]
        qy_ = state[1]
        
        terminate = False
        reward = 0
        new_state = 0
        
        return new_state, reward, terminate
        
    
    def f(self, number):
        return number*'hello world'
    

In [39]:
x = makeMonoEnv()
a,b,c = x.step(1,2)
c

False